In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Splits data into batches of defined size
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

from tqdm import tqdm


In [4]:
file_errors_location = 'ArithOpsTrain.xlsx'
df = pd.read_excel(file_errors_location)

In [6]:
from torchtext.data import get_tokenizer

# Downloads GloVe and FastText
#global_vectors = GloVe(name='840B', dim=300)

# ----------- Text Preprocessing -----------
#nlp = spacy.load("en_core_web_md")
tokenizer = get_tokenizer("basic_english")

In [13]:
dataset = []
vocab, vocab_target = [], []
for idx, row in enumerate(df.iterrows()):
    if idx == 0:
        print(row[1][1], row[1][2], row[1][3], row[1][4], row[1][5])
        continue 
    
    context = tokenizer(row[1][1])
    vocab.extend(context)
    
    ques = tokenizer(row[1][2])
    vocab.extend(ques)

    ans = tokenizer(row[1][3])
    vocab_target.extend(ans)

    ip = [float(x) for x in row[1][4].split()]
    out = float(row[1][5])
    dataset.append((context, ques, ans, ip, out))

Description Question Equation Input Numbers Output


In [16]:
vocab_to_id = {}
ids = 1
for word in vocab:
    if word not in vocab_to_id:
        vocab_to_id[word] = ids
        ids += 1

vocab_target_to_id = {}
idt = 1
for word in vocab_target:
    if word not in vocab_target_to_id:
        vocab_target_to_id[word] = idt
        idt += 1

print(ids, idt)

2617 10


In [64]:
dataset_tokenized = []

for context, ques, ans, ip, out in dataset:
    context_token = []
    for word in context:
        context_token.append(vocab_to_id[word])
    
    ques_token = []
    for word in ques:
        ques_token.append(vocab_to_id[word])

    ans_token = []
    for word in ans:
        ans_token.append(vocab_target_to_id[word])
    
    dataset_tokenized.append((context_token, ques_token, ans_token, ip, out))

In [20]:
dataset_tokenized[0]

([1, 2, 3, 4, 5, 6, 7, 8, 9, 4, 5, 6],
 [10, 11, 12, 13, 14, 15, 4, 5, 16],
 [1, 2, 3, 11],
 [63.0, 50.0],
 113.0)

In [65]:
def split_indices(n, val_pct):

    # Determine size of Validation set
    n_val = int(val_pct * n)

    # Create random permutation of 0 to n-1
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

train_indices, val_indices = split_indices(len(dataset), 0.2)

In [110]:
from torch.nn.utils.rnn import pad_sequence

# ----------- Batching the data -----------
def collate_fn(instn):
    context = [torch.Tensor(x[0]) for x in instn]
    ques = [torch.Tensor(x[1]) for x in instn]
    ans = [torch.Tensor(x[2]) for x in instn]
    ip = [x[3] for x in instn]
    out = [x[4] for x in instn]

    context_pad = pad_sequence(context, batch_first=True, padding_value=10).long()
    ques_pad = pad_sequence(ques, batch_first=True, padding_value=10).long()
    ans_pad = pad_sequence(ans, batch_first=True, padding_value=10).long()

    return (context_pad, ques_pad, ans_pad, ip, out)


batch_size = 64

train_sampler   = SubsetRandomSampler(train_indices)
train_loader    = DataLoader(dataset_tokenized, batch_size, sampler=train_sampler, collate_fn=collate_fn)

val_sampler     = SubsetRandomSampler(val_indices)
val_loader      = DataLoader(dataset_tokenized, batch_size, sampler=val_sampler, collate_fn=collate_fn)

In [111]:
for i in train_loader:
    print(i)
    break

(tensor([[  51,  402,   75,  ...,   10,   10,   10],
        [  33,   52,    3,  ...,   10,   10,   10],
        [2354, 1154,   81,  ...,   10,   10,   10],
        ...,
        [ 136, 1302,    3,  ...,   10,   10,   10],
        [ 182, 1003, 2204,  ...,   10,   10,   10],
        [1660,  128,  129,  ...,   10,   10,   10]]), tensor([[  27,  317,   29,  ...,   10,   10,   10],
        [  27,   28,  325,  ...,   10,   10,   10],
        [  27,  317,  359,  ...,   10,   10,   10],
        ...,
        [  27,   28, 1681,  ...,   10,   10,   10],
        [  10,   11,   12,  ...,   10,   10,   10],
        [  27,   28,   29,  ...,   10,   10,   10]]), tensor([[ 8,  2,  3, 10, 10],
        [ 8,  2,  3, 10, 10],
        [ 1,  1,  2,  3,  5],
        [ 1,  2,  3, 10, 10],
        [ 4,  2,  3, 10, 10],
        [ 1,  4,  2,  3,  5],
        [ 1,  2,  3, 10, 10],
        [ 9,  4,  2,  3,  5],
        [ 4,  5,  3, 10, 10],
        [ 9,  1,  2,  3,  5],
        [ 9,  2,  3, 10, 10],
        [ 4,  4

In [112]:
# ----------- Encoder -----------
class Encoder(nn.Module):
    def __init__(self, rnn_type):
        super().__init__()

        self.embeddings = nn.Embedding(ids+2, 300, padding_idx=0)
        self.dropout = nn.Dropout(0.1)

        if rnn_type == 'gru':
            self.lstm = nn.GRU(input_size = 300, hidden_size = 128, num_layers = 1, batch_first = True, bidirectional = True)
        else:
            self.lstm = nn.LSTM(input_size = 300, hidden_size = 128, num_layers = 1, batch_first = True, bidirectional = True)
    
    def forward(self, Xb):

        x = self.embeddings(Xb)
        x = self.dropout(x)
        out, h = self.lstm(x)

        return out              # B x Seq_len x 128

# ----------- Decoder -----------
class Decoder(nn.Module):
    def __init__(self, rnn_type):
        super().__init__()

        #self.input_combine = nn.Linear(267, 200)

        if rnn_type == 'gru':
            self.gru = nn.GRUCell(input_size = 267, hidden_size = 128)
        else:
            self.gru = nn.LSTMCell(input_size = 256, hidden_size = 128)

        self.rnn_type = rnn_type
            
        self.lin1 = nn.Linear(128, 64)
        self.lin2 = nn.Linear(64, 11)
        
    def forward(self, Cb, sp, y):
        """ Cb - B x 384
            sp - B x seq_len x 128
            y - B x 11
        """

        if self.rnn_type == 'gru':          # TODO: Teacher enforcing
            input = torch.cat((y, Cb), dim = 1) 
            sp = sp[:, 0, :].squeeze(dim=1) 
            #input = self.input_combine(input)
            s = self.gru(input, sp)
        else:
            s = self.gru(Cb)[0]
        x = self.lin1(s)
        x = F.relu(x)
        x = self.lin2(x)

        return x, s

class Translator(nn.Module):
    
    def __init__(self, rnn_type = 'gru'):
        super().__init__()

        self.encoder = Encoder(rnn_type)
        self.attention = nn.Linear(384, 1)   # TODO: Make it 32, 384
        self.decoder = Decoder(rnn_type)

        self.initial_hidden = nn.Linear(256, 128)

        self.transform_hidden_dec = nn.Linear(128, 64)
        self.transform_hidden_enc = nn.Linear(256, 128)
        self.energy = nn.Linear(128, 64)

    def forward(self, Xb, max_len):
        
        # Batch size, Sequence Length
        B, seq_len = Xb.shape[0], Xb.shape[1]

        # Hidden vectors
        H = self.encoder(Xb)                                      # B x seq_len x (128 * 2)
        s = self.initial_hidden(H)                                # TODO: make it learnable  # B x seq_len x 128
        
        logits = torch.zeros(B, 1, idt+1).to(device)
        for i in range(10):
            Hs = torch.cat((H, s), dim = 2)                         # B x seq_len x 384

            # Attention
            a = torch.softmax(self.attention(Hs).squeeze(), dim=1)  # B x seq_len         

            # Context Vectors
            c = torch.bmm(a.unsqueeze(dim = 1), H).squeeze(dim=1)   # B x (128*2)

            x, s = self.decoder(c, s, logits[:, i])          # B x 11
            logits = torch.cat((logits, x.unsqueeze(dim=1)), dim = 1)                                
            s = s.unsqueeze(dim=1)
            s = s.repeat(1, seq_len, 1)
        
        return logits[:, 1:, :]

In [219]:
model = Translator('lstm')

device = torch.device("cuda:0")
model.to(device)
opt = torch.optim.AdamW(model.parameters(), lr=5e-4)
loss_fn = nn.CrossEntropyLoss()

In [220]:
def calculate_metric(ans, ip):
    out = []
    for b in range(len(ans)):
        stack = []
        expression = ans[b]
        # iterate over the string in reverse order
        #print(expression[::-1], ip[b])
        for c in expression[::-1]:
    
            # push operand to stack
            if "number" in c:
                idx = int(c[-1])
                if len(ip[b]) <= idx:
                    stack.append(int(ip[b][0]))
                    continue
                stack.append(int(ip[b][idx]))
    
            elif c in "+-/*":
                # pop values from stack can calculate the result
                # push the result onto the stack again
                try:
                    o1 = stack.pop()
                    o2 = stack.pop()
                    #print(o1, o2)
                except:
                    continue
    
                if c == '+':
                    stack.append(o1 + o2)
    
                elif c == '-':
                    stack.append(o1 - o2)
    
                elif c == '*':
                    stack.append(o1 * o2)
    
                elif c == '/':
                    try:
                        stack.append(o1 / o2)
                    except:
                        print(ans[b], ip[b])
        try:
            out.append(stack.pop())
        except:
            out.append(int(ip[b][0]) + int(ip[b][1]))
    return out


In [221]:
id_to_vocab_target = {}
for i, v in vocab_target_to_id.items():
    id_to_vocab_target[v] = i
id_to_vocab_target[0] = "BOS"
id_to_vocab_target[idt] = "EOS"
#id_to_vocab_target

In [222]:
# ----------- Main Training Loop -----------
max_epoch = 15

best_test_acc = 0
for ep in range(max_epoch):

    epoch_loss = 0

    model.train()
    train_labels = []
    train_pred = []
    correct_train, B_train = 0, 0
    for cont, ques, ans, ip, out in tqdm(train_loader):
        loss = 0
        cont = cont.to(device)
        ques = ques.to(device)
        ans = ans.to(device)

        ans_pred = model(cont, ans.shape[1])

        for i in range(ans.shape[1]):
            loss += loss_fn(ans_pred[:, i], ans[:, i]) 

        loss.backward()
        opt.step()
        opt.zero_grad()

        epoch_loss += float(loss)

        pred = []
        for b in range(ans.shape[0]):
            p = []
            for i in range(ans.shape[1]):
                p.append(id_to_vocab_target[torch.argmax(ans_pred[0,i,:]).item()])
            pred.append(p)
        
        if ep > 5:
            outt = calculate_metric(pred, ip)
            #print(outt)
            for i in range(len(out)):
                #print(outt[i], out[i])
                if float(outt[i]) == float(out[i]):
                    correct_train += 1
                else:
                    #print(outt[i], out[i])
                    pass
            B_train += len(out)       

    print("Epoch: ", ep+1, " Training Loss: ", epoch_loss)
    #print("Train accuracy: ", accuracy_score(train_labels, train_pred)*50)
    if ep > 5:
        print("Train EM: ", (correct_train/ B_train)*100)

    epoch_loss = 0

    model.eval()
    train_labels = []
    train_pred = []
    correct_train, B_train = 0, 0
    for cont, ques, ans, ip, out in tqdm(val_loader):
        loss = 0
        cont = cont.to(device)
        ques = ques.to(device)
        ans = ans.to(device)

        ans_pred = model(cont, ans.shape[1])

        for i in range(ans.shape[1]):
            loss += loss_fn(ans_pred[:, i], ans[:, i]) 

        epoch_loss += float(loss)

        pred = []
        for b in range(ans.shape[0]):
            p = []
            for i in range(ans.shape[1]):
                p.append(id_to_vocab_target[torch.argmax(ans_pred[0,i,:]).item()])
            pred.append(p)
        
        if ep > 5:
            outt = calculate_metric(pred, ip)
            #print(outt)
            for i in range(len(out)):
                #print(outt[i], out[i])
                if float(outt[i]) == float(out[i]):
                    correct_train += 1
                else:
                    #print(outt[i], out[i])
                    pass
            B_train += len(out)       

    print("Epoch: ", ep+1, " Training Loss: ", epoch_loss)
    #print("Train accuracy: ", accuracy_score(train_labels, train_pred)*50)
    if ep > 5:
        print("Train EM: ", (correct_train/ B_train)*100)
    


100%|██████████| 13/13 [00:00<00:00, 33.37it/s]


Epoch:  1  Training Loss:  152.98008632659912


100%|██████████| 4/4 [00:00<00:00, 67.80it/s]


Epoch:  1  Training Loss:  45.846248626708984


100%|██████████| 13/13 [00:00<00:00, 36.70it/s]


...

Epoch:  14  Training Loss:  26.709074020385742
Train EM:  18.974358974358974


100%|██████████| 13/13 [00:00<00:00, 36.93it/s]


Epoch:  15  Training Loss:  96.4856824874878
Train EM:  19.897959183673468


100%|██████████| 4/4 [00:00<00:00, 69.05it/s]

Epoch:  15  Training Loss:  26.39436960220337
Train EM:  18.974358974358974


In [125]:
# ----------- Main Training Loop -----------
max_epoch = 1

best_test_acc = 0
for ep in range(max_epoch):

    epoch_loss = 0

    model.train()
    train_labels = []
    train_pred = []
    correct_train, B_train = 0, 0
    for cont, ques, ans, ip, out in tqdm(train_loader):
        loss = 0
        cont = cont.to(device)
        ques = ques.to(device)
        ans = ans.to(device)

        ans_pred = model(cont, ans.shape[1])

        for i in range(ans.shape[1]):
            loss += torch.mean(loss_fn(ans_pred[:, i], ans[:, i]) * ans[:, i])


        epoch_loss += float(loss)

        for b in range(ans.shape[0]):
            for i in range(ans.shape[1]):
                print(id_to_vocab_target[torch.argmax(ans_pred[b,i,:]).item()], end=" ")
                print(id_to_vocab_target[ans[b, i].item()])
            print()
        #y_hat = torch.softmax(y_hat, dim = 1).argmax(dim=1)

        #correct, B = exact_match(y_hat, yb)
        #correct_train += correct
        #B_train += B
        #train_labels.extend(yb.cpu().detach().numpy())
        #train_pred.extend(y_hat.cpu().detach().numpy())

    print("Epoch: ", ep+1, " Training Loss: ", epoch_loss)
    #print("Train accuracy: ", accuracy_score(train_labels, train_pred)*100)
    #print("Train EM: ", (correct_train/ B_train)*100)


 15%|█▌        | 2/13 [00:00<00:01, 10.81it/s]

+ /
number0 number0
number2 number2
EOS EOS
EOS EOS

+ /
+ -
number0 number0
number1 number2
number2 number1

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

+ /
number0 number0
number1 number1
EOS EOS
EOS EOS

+ +
+ -
number0 number0
number1 number1
number2 number2

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number2
EOS EOS
EOS EOS

- -
number0 number1
number1 number0
EOS EOS
EOS EOS

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

* +
number0 number0
number1 number1
EOS EOS
EOS EOS

- /
number0 number0
number1 number1
EOS EOS
EOS EOS

+ -
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number1
number1 number3
EOS EOS
EOS EOS

+ *
+ -
number0 number1
number1 number2
number2 number0

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

+ +
+ +
number0 number0
number1 nu

 31%|███       | 4/13 [00:00<00:00, 10.98it/s]

-
number0 number0
number1 number1
number2 number2

- /
- -
number0 number0
number1 number1
EOS number2

- *
number0 number0
number1 number1
EOS EOS
EOS EOS

+ /
number0 number1
number1 number0
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

+ /
number0 number0
number1 number1
EOS EOS
EOS EOS

- *
number0 /
number1 number1
number2 number0
EOS number2

- *
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- *
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
+ +
number0 number0
number1 number1
number2 number2

- -
- -
number0 number0
number1 number1
number2 number2

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

+ *
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- +
+ -
number0 number0
number1 nu

 46%|████▌     | 6/13 [00:00<00:00, 11.04it/s]

...

EOS

- /
number0 number0
number1 number2
EOS EOS
EOS EOS

- -
number0 number1
number1 number0
EOS EOS
EOS EOS

+ +
+ -
number0 number2
number1 number1
number2 number0

- -
number0 number1
number1 number0
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
+ -
number0 number0
number1 number1
number2 number2

+ +
number1 number2
number2 number3
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
+ -
number0 number0
number1 number1
number2 number2

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

+ -
+ -
number0 number0
number1 number1
number2 number2

- -
number0 number1
EOS number0
EOS EOS
EOS EOS

+ *
number0 number0
number1 number1
EOS EOS
EOS EOS

- +
number0 number5
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number3
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

+ +
number0 number0
number1 number1
EOS EOS
EOS EOS

/ /
number0 number0
number1 n

 77%|███████▋  | 10/13 [00:00<00:00, 10.98it/s]

number1 number1
EOS EOS
EOS EOS

+ +
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

+ +
number0 +
number1 number1
number2 number2
EOS number3

- +
+ -
number0 number0
number1 number2
number2 number3

* *
number0 number0
number1 number1
EOS EOS
EOS EOS

- *
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 *
number2 number2
number2 number1

- /
number0 number0
number1 number1
EOS EOS
EOS EOS

+ +
+ +
number0 number0
number1 number1
number2 number2

/ /
number0 number0
number1 number1
EOS EOS
EOS EOS

+ +
number0 number0
number1 number1
EOS EOS
EOS EOS

+ *
+ /
number0 number1
number1 number0
number2 number2

- /
+ +
number0 number0
number1 number1
number2 number2

- *
number0 number0
number1 number1
EOS EOS
EOS EOS

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

- *
number0 number0
number1 number

100%|██████████| 13/13 [00:01<00:00, 11.55it/s]

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number1
number1 number0
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
+ number0
number0 *
number2 number2
number2 number1

+ *
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
- -
number0 number0
number1 number1
number2 number2

+ +
+ +
number0 number0
number1 number1
number2 number2

- -
number0 number1
number1 number0
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

- /
number0 number0
number1 number2
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

+ *
number0 number0
number1 number1
EOS EOS
EOS EOS

- -
number0 number0
number1 number1
EOS EOS
EOS EOS

+ +
+ +
number0 number2
number1 number1
number2 number0

+ +
number0 number0
number1 number1
EOS EOS
EOS EOS

- +
number0 number0
number1 number1
EOS EOS
EOS EOS

+ -
number0 number1
number2 nu